In [1]:
import torch
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import transformers
import numpy as np
from os import path
import pandas as pd
from constants import *
import dataset_utils as du
from new_model import create_model, create_model_no_stackx, create_model_only_stackx
from training_metrics import TrainMetrics
from evaluation_metrics import EvalMetrics

In [2]:
def get_next_name_train(name):
    pathstr = name
    i = 1
    while path.exists(pathstr + '-train.json'):
        pathstr = name + str(i)
        i += 1
    return pathstr

def get_next_name_eval(name):
    pathstr = name
    i = 1
    while path.exists(pathstr + '-eval.json'):
        pathstr = name + str(i)
        i += 1
    return pathstr

In [3]:
# Uncomment the following line to rerun pre-processing and tokenization
# prepare_and_save(RAW_DATASET_PATH, path.join(PROCESSED_DIR, PROCESSED_DATASET))

data = pd.read_pickle(path.join(PROCESSED_DIR, PROCESSED_DATASET))
interest = du.get_interest_dataset(data) 
interest = du.get_balanced_dataset(interest, 'interest_binary', random_state=1)
interest = interest.sample(frac=1, random_state=1)


/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/home/ubuntu/anaconda3/envs/tensorflow_p37/lib/python3.7/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [4]:
# K-fold split
features = du.get_features(interest)
labels = du.get_labels(interest, 'interest_binary')
folds = du.kfold_split(features, labels, num_folds=8)




In [ ]:
# Hyperparameters
INITIAL_ALPHA = 1e-5
LEARNING_RATE = tf.keras.optimizers.schedules.ExponentialDecay(INITIAL_ALPHA, decay_steps=100, decay_rate=0.9, staircase=True)
BATCH_SIZE = 2
EPOCHS = 6

# train_set, dev_set, test_set = du.split_datasets(interest, TRAIN_RATIO, DEV_RATIO)
# train_features, train_labels = du.get_features(train_set), du.get_labels(train_set, 'interest_binary')
# dev_features, dev_labels = du.get_features(dev_set), du.get_labels(dev_set, 'interest_binary')
# test_features, test_labels = du.get_features(test_set), du.get_labels(test_set, 'interest_binary')

train_features = folds[2]['train_features']
train_labels = folds[2]['train_labels']


name = get_next_name_train('metrics/model_overfit')
training_metrics = TrainMetrics(name, epochs=EPOCHS)

model = create_model_no_stackx(MAX_LEN, NUM_SX_FEATURES)

model.compile(optimizer=keras.optimizers.Adam(LEARNING_RATE), loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])


hist = model.fit(train_features, train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, shuffle=True, callbacks=[training_metrics])

training_metrics.plot_loss()
training_metrics.plot_accuracy()
#training_metrics.save_metrics_json()



Some layers from the model checkpoint at xlnet-base-cased were not used when initializing TFXLNetForSequenceClassification: ['lm_loss']
- This IS expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFXLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj', 'sequence_summary']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/6
178/348 [==============>...............] - ETA: 1:30 - loss: 0.7049 - accuracy: 0.4547

In [ ]:
dev_features = folds[2]['test_features']
dev_labels = folds[2]['test_labels']

dev_preds = model.predict(dev_features, batch_size=BATCH_SIZE)
dev_preds = np.rint(dev_preds).astype(int)
eval_dataset = {'feats': dev_features, 'labels': dev_labels, 'preds': dev_preds}

name = get_next_name_eval('metrics/model_overfit')
eval_metrics = EvalMetrics(name, dataset=eval_dataset)
eval_metrics.plot_confusion_matrix()
print(f'F1 score {eval_metrics.get_f1score()}')
print(f'Accuracy {eval_metrics.get_accuracy()}')


In [22]:
# training_metrics.save_metrics_json()
# eval_metrics.save_metrics_json()
# import os
#os.mkdir('saved_model_weights/FILL_IN')
#model.save_weights('saved_model_weights/FILL_IN/v1')

In [8]:
print(f'F1 score{eval_metrics.get_f1score()}')
print(f'Accuracy{eval_metrics.get_accuracy()}')

F1 score0.49638947123223853
Accuracy0.559748427672956


In [ ]:
training_metrics = TrainMetrics('metrics/model_overfit5', load_json=True)
training_metrics.plot_accuracy()
training_metrics.plot_accuracy()

In [26]:
hist.history

{'loss': [0.2950187623500824], 'accuracy': [0.8936781883239746]}